In [74]:
pip install snorkel

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [75]:
import pandas as pd
import random

In [76]:
tweet = pd.read_csv('TWEETSS.csv')

In [77]:
tweet

,Unnamed: 0,Date,User,Tweet
0,0,2022-12-31 23:11:28+00:00,BevonKimutai,@DanOjowa When Uhuru Kenyatta appointed Moody ...
1,1,2022-12-31 21:59:22+00:00,Dacha37533500,@edutheguru @PolycarpHinga Kwenda huko. Reject...
2,2,2022-12-31 21:47:40+00:00,kerubobree1,@Nesh_maina I miss Uhuru Kenyatta
3,3,2022-12-31 21:27:19+00:00,Lewisto90476962,@citizentvkenya This too much I think this reg...
4,4,2022-12-31 21:26:04+00:00,SaSamnjuguna16,@OleItumbi @WilliamsRuto @OleItumbi just to le...
...,...,...,...,...
4995,4995,2022-12-02 02:18:53+00:00,RGakwerere,"President Uhuru Kenyatta further noted, “We do..."
4996,4996,2022-12-02 02:18:52+00:00,RGakwerere,President Uhuru Kenyatta: “We have no problem ...
4997,4997,2022-12-01 22:21:57+00:00,gchahal,JUST IN: #BNNDRC Reports. \n\nThe work related...
4998,4998,2022-12-01 22:18:17+00:00,FatakiHugo,"@afrikarabia Bravo Uhuru Kenyatta, Kagame doit..."


In [78]:
print(tweet.shape)

(5000, 4)


In [79]:
#Define our labeling function
from snorkel.labeling import labeling_function,PandasLFApplier,LFAnalysis

In [80]:
#Constants for our labels
Irrelevant = 1
Relevant = 0
Unknown = -1

In [111]:
###Keyword_Lookup
@labeling_function()
def lf_keyword_lookup(x):
    keywords = ["whatsapp","size","retweet","order","ksh","kshs","price","offer","affordable","shop","prices","toyota","prado"]
    return Irrelevant if any(word in x.Tweet.lower() for word in keywords)else Relevant

In [112]:
###Pattern Lookup/Regex Fxn
import re
@labeling_function()
def lf_regex_contains_call(x):
    return Irrelevant if re.search(r"call.*whatsapp",x.Tweet,flags=re.I)else Relevant

In [113]:
###Pattern Lookup/Regex Fxn
import re
@labeling_function()
def lf_regex_contains_price(x):
    return Irrelevant if re.search(r"price.*call",x.Tweet,flags=re.I)else Relevant

In [114]:
###Apply Fxn
lfs = [lf_keyword_lookup,lf_regex_contains_call,lf_regex_contains_price]
applier=PandasLFApplier(lfs=lfs)
L_train = applier.apply(tweet)

100%|██████████| 5000/5000 [00:00<00:00, 6192.51it/s]


In [115]:
#Label Matrix
L_train

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [116]:
###Evaluate the Performance
LFAnalysis(L=L_tweet,lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_keyword_lookup,0,[1],0.1128,0.0088,0.0
lf_regex_contains_call,1,[1],0.0084,0.0084,0.0
lf_regex_contains_price,2,[1],0.0004,0.0004,0.0


In [117]:
###LabelModel
from snorkel.labeling.model import MajorityLabelVoter,LabelModel

In [118]:
#Build the model
label_model = LabelModel(cardinality=2,verbose=True)
label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
 40%|███▉      | 199/500 [00:00<00:00, 991.66epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.001]
INFO:root:[300 epochs]: TRAIN:[loss=0.001]
100%|██████████| 500/500 [00:00<00:00, 1019.10epoch/s]
INFO:root:Finished Training


In [119]:
###Unlabeled Dataset
tweet.head()

,Unnamed: 0,Date,User,Tweet,Intention
0,0,2022-12-31 23:11:28+00:00,BevonKimutai,@DanOjowa When Uhuru Kenyatta appointed Moody ...,0
1,1,2022-12-31 21:59:22+00:00,Dacha37533500,@edutheguru @PolycarpHinga Kwenda huko. Reject...,0
2,2,2022-12-31 21:47:40+00:00,kerubobree1,@Nesh_maina I miss Uhuru Kenyatta,0
3,3,2022-12-31 21:27:19+00:00,Lewisto90476962,@citizentvkenya This too much I think this reg...,0
4,4,2022-12-31 21:26:04+00:00,SaSamnjuguna16,@OleItumbi @WilliamsRuto @OleItumbi just to le...,0


In [120]:
#apply our label model on our unlabeled dataset
tweet['Intention'] = label_model.predict(L=L_train,tie_break_policy="abstain")

In [121]:
tweet

,Unnamed: 0,Date,User,Tweet,Intention
0,0,2022-12-31 23:11:28+00:00,BevonKimutai,@DanOjowa When Uhuru Kenyatta appointed Moody ...,0
1,1,2022-12-31 21:59:22+00:00,Dacha37533500,@edutheguru @PolycarpHinga Kwenda huko. Reject...,0
2,2,2022-12-31 21:47:40+00:00,kerubobree1,@Nesh_maina I miss Uhuru Kenyatta,0
3,3,2022-12-31 21:27:19+00:00,Lewisto90476962,@citizentvkenya This too much I think this reg...,0
4,4,2022-12-31 21:26:04+00:00,SaSamnjuguna16,@OleItumbi @WilliamsRuto @OleItumbi just to le...,0
...,...,...,...,...,...
4995,4995,2022-12-02 02:18:53+00:00,RGakwerere,"President Uhuru Kenyatta further noted, “We do...",0
4996,4996,2022-12-02 02:18:52+00:00,RGakwerere,President Uhuru Kenyatta: “We have no problem ...,0
4997,4997,2022-12-01 22:21:57+00:00,gchahal,JUST IN: #BNNDRC Reports. \n\nThe work related...,0
4998,4998,2022-12-01 22:18:17+00:00,FatakiHugo,"@afrikarabia Bravo Uhuru Kenyatta, Kagame doit...",0


In [122]:
tweet['Intention'].value_counts()

0    4714
1     286
Name: Intention, dtype: int64

In [123]:
tweet

,Unnamed: 0,Date,User,Tweet,Intention
0,0,2022-12-31 23:11:28+00:00,BevonKimutai,@DanOjowa When Uhuru Kenyatta appointed Moody ...,0
1,1,2022-12-31 21:59:22+00:00,Dacha37533500,@edutheguru @PolycarpHinga Kwenda huko. Reject...,0
2,2,2022-12-31 21:47:40+00:00,kerubobree1,@Nesh_maina I miss Uhuru Kenyatta,0
3,3,2022-12-31 21:27:19+00:00,Lewisto90476962,@citizentvkenya This too much I think this reg...,0
4,4,2022-12-31 21:26:04+00:00,SaSamnjuguna16,@OleItumbi @WilliamsRuto @OleItumbi just to le...,0
...,...,...,...,...,...
4995,4995,2022-12-02 02:18:53+00:00,RGakwerere,"President Uhuru Kenyatta further noted, “We do...",0
4996,4996,2022-12-02 02:18:52+00:00,RGakwerere,President Uhuru Kenyatta: “We have no problem ...,0
4997,4997,2022-12-01 22:21:57+00:00,gchahal,JUST IN: #BNNDRC Reports. \n\nThe work related...,0
4998,4998,2022-12-01 22:18:17+00:00,FatakiHugo,"@afrikarabia Bravo Uhuru Kenyatta, Kagame doit...",0


In [124]:
tweet. to_csv("C:/Users/Maria Gichomo/Desktop/labeledtweets.csv")